In [101]:
import tensorflow as tf
import numpy as np
from tensorflow.contrib import rnn
import pprint
pp=pprint.PrettyPrinter(indent=4)
tf.reset_default_graph()
sess = tf.InteractiveSession()
#Session()와 거의 유사, 정의된 노드를 seesion 객체 eval 함수 수행 

C:\Users\B-7\Anaconda3\lib\site-packages\tensorflow\python\client\session.py:1702: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [102]:
h=[1,0,0,0]
e=[0,1,0,0]
l=[0,0,1,0]
o=[0,0,0,1]

In [103]:
with tf.variable_scope('one_cell') as scope:
    hidden_size=2
    cell=tf.contrib.rnn.BasicRNNCell(num_units=hidden_size)
    print(cell.output_size, cell.state_size)
    xdata=np.array([[h]], dtype=np.float32)
    pp.pprint(xdata)
    output, _states = tf.nn.dynamic_rnn(cell, xdata, dtype=tf.float32)
    sess.run(tf.global_variables_initializer())
    pp.pprint(output.eval())


2 2
array([[[1., 0., 0., 0.]]], dtype=float32)
array([[[ 0.38746822, -0.4406506 ]]], dtype=float32)


In [104]:
with tf.variable_scope('two_sequences') as scope:
    hidden_size=2
    cell=tf.contrib.rnn.BasicRNNCell(num_units=hidden_size)
    xdata=np.array([[h,e,l,l,o]], dtype=np.float32)
    print(xdata.shape)
    pp.pprint(xdata)
    outputs, states=tf.nn.dynamic_rnn(cell, xdata, dtype=tf.float32)
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())

(1, 5, 4)
array([[[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]]], dtype=float32)
array([[[ 0.6038574 ,  0.08811066],
        [-0.13696282, -0.43274224],
        [ 0.5921835 , -0.19798875],
        [ 0.81095856,  0.2879693 ],
        [-0.17979166, -0.06437916]]], dtype=float32)


In [105]:
with tf.variable_scope('3_batch') as scope:
    hidden_size=2
    cell=tf.contrib.rnn.BasicLSTMCell(num_units=hidden_size, state_is_tuple=True)
    xdata=np.array([[h,e,l,l,o],
                    [e,o,l,l,l],
                    [l,l,e,e,l]], dtype=np.float32)
    print(xdata.shape)
    pp.pprint(xdata)
    outputs, states=tf.nn.dynamic_rnn(cell, xdata, dtype=tf.float32)
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())

(3, 5, 4)
array([[[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]],

       [[0., 1., 0., 0.],
        [0., 0., 0., 1.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.]],

       [[0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.]]], dtype=float32)
array([[[ 0.08085825,  0.10402066],
        [-0.04780198,  0.16311891],
        [ 0.07428766,  0.2790487 ],
        [ 0.14849658,  0.3723005 ],
        [ 0.09775919,  0.3016987 ]],

       [[-0.12761089,  0.06537931],
        [-0.10158084,  0.04547361],
        [ 0.03099267,  0.17395328],
        [ 0.11970399,  0.28412837],
        [ 0.17162664,  0.37545094]],

       [[ 0.08969265,  0.12636957],
        [ 0.14760025,  0.24278697],
        [ 0.02270011,  0.2634428 ],
        [-0.0734803 ,  0.28904766],
        [ 0.06768489,  0.38966534]]], dtype=float32)


In [108]:
tf.reset_default_graph()

#hihell -> ihello
idx2char = ['h', 'i', 'e', 'l', 'o']
x_data = [[0, 1, 0, 2, 3, 3]]   # hihell => 102334
x_one_hot = [[[1, 0, 0, 0, 0],   # h 0
              [0, 1, 0, 0, 0],   # i 1
              [1, 0, 0, 0, 0],   # h 0
              [0, 0, 1, 0, 0],   # e 2
              [0, 0, 0, 1, 0],   # l 3
              [0, 0, 0, 1, 0]]]  # l 3

y_data = [[1, 0, 2, 3, 3, 4]]    # ihello

input_dim = 5  # one-hot size
hidden_size = 5  # output from the LSTM. 
batch_size = 1   # one sentence
sequence_length = 6  # ihello == 6
learning_rate = 0.1

X = tf.placeholder(
    tf.float32, [None, sequence_length, input_dim])  # X one-hot
Y = tf.placeholder(tf.int32, [None, sequence_length])  # Y label

cell = tf.contrib.rnn.BasicLSTMCell(num_units=hidden_size, state_is_tuple=True)
initial_state = cell.zero_state(batch_size, tf.float32)
outputs, _states = tf.nn.dynamic_rnn(
    cell, X, initial_state=initial_state, dtype=tf.float32)

# FC layer
X_for_fc = tf.reshape(outputs, [-1, hidden_size])
outputs = tf.contrib.layers.fully_connected(
    inputs=X_for_fc, num_outputs=5, activation_fn=None)

outputs = tf.reshape(outputs, [batch_size, sequence_length, 5])

weights = tf.ones([batch_size, sequence_length])
cost = tf.contrib.seq2seq.sequence_loss(
    logits=outputs, targets=Y, weights=weights)
rcost = tf.reduce_mean(cost)
train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(rcost)

prediction = tf.argmax(outputs, axis=2)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(50):
        l, _ = sess.run([rcost, train], feed_dict={X: x_one_hot, Y: y_data})
        result = sess.run(prediction, feed_dict={X: x_one_hot})
        print(i, "cost:", l, "prediction: ", result, "true Y: ", y_data)
        
        res_str=[idx2char[c] for c in np.squeeze(result)]
        print("Wt.예측 문자열: ", "".join(res_str))

0 cost: 1.6303952 prediction:  [[3 3 3 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]
Wt.예측 문자열:  llllll
1 cost: 1.5466766 prediction:  [[3 3 3 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]
Wt.예측 문자열:  llllll
2 cost: 1.4876993 prediction:  [[3 3 3 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]
Wt.예측 문자열:  llllll
3 cost: 1.4339031 prediction:  [[3 3 3 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]
Wt.예측 문자열:  llllll
4 cost: 1.3666452 prediction:  [[3 3 3 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]
Wt.예측 문자열:  llllll
5 cost: 1.2775841 prediction:  [[2 3 2 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]
Wt.예측 문자열:  elelll
6 cost: 1.162534 prediction:  [[2 3 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
Wt.예측 문자열:  elello
7 cost: 1.0274603 prediction:  [[1 3 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
Wt.예측 문자열:  ilello
8 cost: 0.88003063 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
Wt.예측 문자열:  ihello
9 cost: 0.73032916 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
Wt.예측 문자열:  ihello
10 cost: 0.59424514 prediction:  [[1 0 

In [122]:
#squeeze(차원 감소) <-> expanddims(차원 증가)
x=np.array([1,2])
x.shape
y=np.expand_dims(x, axis=1) #2행 1열
y
y=np.expand_dims(x, axis=0) #1행 2열
y

x=np.array([[[0],[1],[2]]])
x.shape #(1,3,1)
np.squeeze(x, axis=0) #(3,1)
np.squeeze(x, axis=2) #(1,3) 


x=np.array([[[0],[1],[2]]])
x=np.squeeze(x) #array([0,1,2])
x
np.shape(x) #(3,)

(3,)